In [ ]:
!nvidia-smi

Wed Sep 29 06:32:23 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla V100-PCIE...  Off  | 00000000:18:00.0 Off |                    0 |
| N/A   43C    P0    42W / 250W |   1181MiB / 32510MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla V100-PCIE...  Off  | 00000000:AF:00.0 Off |                    0 |
| N/A   

In [3]:
from argparse import Namespace
import logging
opt = {
    "batch_size": 4,
    "num_workers": 4,
    "lr": 1e-4,
    "max_epochs": 10,
    "warmup_ratio": 0.2,
    "print_step": 100,
    "save_path": "drive/MyDrive/model_weights",
} 
args = Namespace(**opt)

In [4]:
import json
path = "drive/MyDrive/"
train_file = path + "train.json"
with open(train_file, "r") as f:
    TRAIN_DATA = json.load(f)

In [5]:
import json
path = "drive/MyDrive/"
test_file = path + "test.json"
with open(test_file, "r") as f:
    TEST_DATA = json.load(f)

In [6]:
!pip install git+https://github.com/SKT-AI/KoBART#egg=kobart

  Cloning https://github.com/SKT-AI/KoBART to /tmp/pip-install-1axfx_r3/kobart_e6663006222c413fbeb45cb7639de8e8
  Running command git clone -q https://github.com/SKT-AI/KoBART /tmp/pip-install-1axfx_r3/kobart_e6663006222c413fbeb45cb7639de8e8
     |████████████████████████████████| 1.9 MB 4.0 MB/s 
     |████████████████████████████████| 776.8 MB 15 kB/s 
     |████████████████████████████████| 895 kB 56.8 MB/s 
     |████████████████████████████████| 3.3 MB 48.7 MB/s 
  Created wheel for kobart: filename=kobart-0.4-py3-none-any.whl size=8538 sha256=fb52a693e2bfe94dc3bdbd7c297d084c108254a7090fec8d4d528c9a956b1663
  Stored in directory: /tmp/pip-ephem-wheel-cache-ommlj1yd/wheels/6e/55/c4/bd4fede223bc304089ac8da2a2099a69db3fcd4b0e853383f5
Successfully built kobart
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into 

In [7]:
from kobart import get_kobart_tokenizer
kobart_tokenizer = get_kobart_tokenizer()
kobart_tokenizer.tokenize("안녕하세요. 한국어 BART 입니다.🤣:)l^o")

[██████████████████████████████████████████████████]


['▁안녕하', '세요.', '▁한국어', '▁B', 'A', 'R', 'T', '▁입', '니다.', '🤣', ':)', 'l^o']

In [8]:
from transformers import BartModel, BartForConditionalGeneration
from kobart import get_pytorch_kobart_model, get_kobart_tokenizer
kobart_tokenizer = get_kobart_tokenizer()
model = BartForConditionalGeneration.from_pretrained(get_pytorch_kobart_model())#BartModel.from_pretrained(get_pytorch_kobart_model())
inputs = kobart_tokenizer(['안녕하세요.'], return_tensors='pt')
model(inputs['input_ids'])

using cached model
[██████████████████████████████████████████████████]


Seq2SeqLMOutput([('logits',
                  tensor([[[-3.1152,  5.6895, -8.4704,  ..., -7.6067, -4.6915, -4.2773],
                           [-6.7131,  4.5574, -7.6131,  ..., -6.8429, -5.2066, -6.7749]]],
                         grad_fn=<AddBackward0>)),
                 ('past_key_values',
                  ((tensor([[[[-9.7980e-02, -6.6584e-01, -1.8089e+00,  ...,  9.6023e-01,
                               -1.8818e-01, -1.3252e+00],
                              [-6.2507e-01,  5.1009e-01, -7.4878e-01,  ...,  8.6230e-01,
                                1.5722e-01, -6.0267e-01]],
                    
                             [[ 5.4597e-01, -2.3990e-01,  1.5901e+00,  ...,  4.3655e-01,
                                7.9514e-01,  8.9880e-02],
                              [-1.7327e-01, -6.3167e-01,  4.5152e-02,  ..., -1.4111e-01,
                                1.8678e-01, -1.2081e-01]],
                    
                             [[ 1.4621e+00,  1.8980e+00, -7.6696e-01,  .

In [9]:
import pandas as pd
train = pd.DataFrame(columns=['uid', 'title', 'region', 'context', 'summary'])
uid = 1000
for data in TRAIN_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        train.loc[uid, 'uid'] = uid
        train.loc[uid, 'title'] = data['title']
        train.loc[uid, 'region'] = data['region']
        train.loc[uid, 'context'] = context[:-1]
        train.loc[uid, 'summary'] = data['label'][agenda]['summary']
        uid += 1

test = pd.DataFrame(columns=['uid', 'title', 'region', 'context'])
uid = 2000
for data in TEST_DATA:
    for agenda in data['context'].keys():
        context = ''
        for line in data['context'][agenda]:
            context += data['context'][agenda][line]
            context += ' '
        test.loc[uid, 'uid'] = uid
        test.loc[uid, 'title'] = data['title']
        test.loc[uid, 'region'] = data['region']
        test.loc[uid, 'context'] = context[:-1]
        uid += 1

In [10]:
train['total'] = train.title + ' ' + train.region + ' ' + train.context
test['total'] = test.title + ' ' + test.region + ' ' + test.context


In [11]:
df_train = train.iloc[:-200]
df_val = train.iloc[-200:]

In [46]:
import os
import glob
import torch
import ast
import numpy as np
import pandas as pd
from tqdm import tqdm, trange
from torch.utils.data import Dataset, DataLoader, IterableDataset

class KoBARTSummaryDataset(Dataset):
    def __init__(self, file, tok, max_len=512, pad_index = 0, ignore_index=-100, train=True):
        super().__init__()
        self.tok = tok
        self.max_len = max_len
        self.docs = file
        self.len = len(self.docs)
        self.pad_index = pad_index
        self.ignore_index = ignore_index
        self.train = train

    def add_padding_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.pad_index] *(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            i=0
            l=np.zeros(((len(inputs)//self.max_len),self.max_len))
            while ((len(inputs)//self.max_len)-i)>0:           
                  l[i] = inputs[i*(self.max_len):(i+1)*(self.max_len)]
                  i+=1
            inputs=l
        return inputs

    def add_ignored_data(self, inputs):
        if len(inputs) < self.max_len:
            pad = np.array([self.ignore_index] *(self.max_len - len(inputs)))
            inputs = np.concatenate([inputs, pad])
        else:
            inputs = inputs[:self.max_len]
        return inputs
    
    def __getitem__(self, idx):
        instance = self.docs.iloc[idx]
        context = instance['context']
        if self.train:
            summary = instance['summary']
        input_ids = self.tok.encode(context)
        input_ids = self.add_padding_data(input_ids)

        if self.train:
            label_ids = self.tok.encode(instance['summary'])
            label_ids.append(self.tok.eos_token_id)
            dec_input_ids = [self.pad_index]
            dec_input_ids += label_ids[:-1]
            dec_input_ids = self.add_padding_data(dec_input_ids)
            label_ids = self.add_ignored_data(label_ids)


            return {'input_ids': np.array(input_ids, dtype=np.int_),
                    'decoder_input_ids': np.array(dec_input_ids, dtype=np.int_),
                    'labels': np.array(label_ids, dtype=np.int_)}
        else:
            return {'input_ids': np.array(input_ids, dtype=np.int_),}
    
    def __len__(self):
        return self.len

In [47]:
test_dataset = KoBARTSummaryDataset(file=df_val, tok=kobart_tokenizer, train=False)
test_dataloader = DataLoader(test_dataset, batch_size=1, num_workers=args.num_workers, shuffle=False)

In [45]:
test_dataset[-1]

[29072. 17208. 23361. 14789. 11900. 11280. 16669. 29072. 17208.  9120.
 14081.  9123. 23381.  9866. 14459. 12141. 12258. 11900. 22316. 18664.
 14581. 17770. 29883. 18115. 14979. 20473. 16499. 12005. 14092. 21102.
 14640. 12013. 14633. 15449. 13672. 13724.   243. 14459. 12141. 14979.
 22316. 14573. 24991. 14581. 17770. 29883. 14338.  1834. 14745.   249.
 14515.  9866. 14979. 22316. 14573.  1835. 14062. 10339. 11786. 14131.
 12013.   240. 21916. 29072. 17208.  9120. 14081.  9123. 23381.  9866.
 14979. 22316. 14573. 29244. 16571. 29883. 29244. 14172. 12002. 17512.
 25925. 26638. 14410.  9743. 14158. 23770. 25925. 14330. 21888.  9264.
 25490. 14562. 17236. 13699. 14410.  9743. 25925. 14330. 21888.  9264.
 25490. 14562. 23433.  9743. 14490. 18940.  9694. 14218. 14032.  9102.
 20032. 15170. 14850. 12819. 13468. 12344. 12612. 18448. 29072. 17208.
  9120. 14058. 23807. 14080.  9495. 17784. 14104.   243. 18068. 14058.
 23807. 17596. 14880.  9120. 14030. 15090. 14058. 14026. 17327. 16574.
 22879

{'input_ids': array([[    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [    0,     0,     0, ...,     0,     0,     0],
        [14033, 14058, 21353, ..., 19686, 12060, 25742]])}

In [14]:
import torch
import torch.nn as nn
class KobartSummaryModule(nn.Module):
    def __init__(self, device):
        super().__init__()
        self.device = device
        self.model = BartForConditionalGeneration.from_pretrained(get_pytorch_kobart_model()).to(self.device)
        
    def forward(self, input, train=True):
        input_ids = torch.as_tensor(input["input_ids"]).to(self.device)
        if train:
            decoder_input_ids = torch.as_tensor(input["decoder_input_ids"]).to(self.device)
            labels = torch.as_tensor(input["labels"]).to(self.device)

            if len(input_ids.shape) == 1:
                input_ids, decoder_input_ids, labels = input_ids.unsqueeze(0), decoder_input_ids.unsqueeze(0), labels.unsqueeze(0)
            
            return_dict = self.model(input_ids=input_ids, decoder_input_ids=decoder_input_ids, labels=labels) 

            loss = return_dict["loss"]

            return loss, return_dict
        else:
            if len(input_ids.shape) == 1:
                input_ids = input_ids.unsqueeze(0)
                
            return_dict = self.model(input_ids=input_ids)

            return return_dict

    def generate(self, input, num_beams=4, max_length=50, early_stopping=True):
        input_ids = torch.as_tensor(input["input_ids"]).to(self.device)
        if len(input_ids.shape) == 1:
                input_ids = input_ids.unsqueeze(0)

        return self.model.generate(input_ids, num_beams=num_beams, max_length=max_length, early_stopping=early_stopping)

        

        


In [15]:
device = "cuda"
model = KobartSummaryModule(device=device)
model.load_state_dict(torch.load(args.save_path + "/model_1.pth"))
#return_dict = model(input, train=False)
#logits = return_dict["logits"]

using cached model


<All keys matched successfully>

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#summary_ids = model.generate(input, num_beams=4, max_length=100, early_stopping=True)
#print([kobart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids])

In [16]:
def decode_summary(summary_ids):
    return [kobart_tokenizer.decode(g, skip_special_tokens=True, clean_up_tokenization_spaces=False) for g in summary_ids]


In [21]:
def summarization(i):
    context = df_val.iloc[i]["context"]
    input = test_dataset[i]
    summary_ids = model.generate(input, num_beams=4, max_length=100, early_stopping=True)
    summary = decode_summary(summary_ids)

    return{"context": context, "summary": summary}

In [ ]:
stride=20
length=500
for i in range(test.shape[0])
   if len(test.iloc[i]["context"])>700
      for j in range((test.iloc[i]["context"]-length)//stride))
         context = test.iloc[i]["context"][(j*stride):(j*stride)+length]
         input=

In [55]:
stride=20
length=500
j=0
print(test.iloc[0]["context"][(j*stride):(j*stride)+length])
print(len(test.iloc[0]["context"]))
print(len(test_dataset[0]['input_ids']))


의석을 정돈하여 주시기 바랍니다. 성원이 되었으므로 지금부터 음성군의회 제235회 제1차 정례회 제1차 본회의를 개의하겠습니다. 먼저 의회사무과장으로부터 보고가 있겠습니다. 의회사무과장입니다. 먼저 지난 제234회 임시회 제1차 본회의에서 의결하여 이송한 안건에 대해서 보고를 드리겠습니다. 2012년 5월 29일 제1차 본회의에서 의결하여 이송한 음성군 지방공무원 정원 조례 일부개정조례안은 2012년 6월 15일자로 공포되었습니다. 다음은 제235회 제1차 정례회 제1차 본회의와 관련하여 보고 드리겠습니다. 「지방자치법」제44조 및 「음성군 회기와 그 운영 등에 관한 조례」제4조제1항에 따라 정례회를 개최하고자 6월 10일자로 집회공고를 하였습니다. 6월 14일 이대웅 의원님 외 2분의 의원님으로 환경분야 현지확인 특별위원회 구성 결의안이, 손달섭 의원님 외 2분의 의원님으로부터 제234회 임시회 시 활동한 주요사업 현지확인 결과 보고의 건이 각각 접수되었습니다. 음성군수로부터 20
630
512


In [49]:
summarization(199)

{'context': '축산식품과장 송요성입니다. 축산식품과 소관 2020년도 군정주요 현안사업을 보고드리겠습니다. 상반기 주요업무 실적은 서면으로 갈음하고, 기본현황, 군정 주요 현안사업 순으로 보고드리겠습니다. (「2020년도 주요 현안사업」 부록에 실음) 이상으로 축산식품과 소관 2020년도 주요 현안사업 보고를 마치겠습니다. 보고를 들으시고 질의하실 의원님 계시면 질의하여 주시기 바랍니다. 김영호 의원님 질의하여 주시기 바랍니다. 과장님 설명하시느라고 수고하셨습니다. 악취포집차량은 축산식품과하고 관련이 없나요? 예, 저희하고 관련이 없고 환경과에서 관리하고 있습니다, 청소위생과하고. 악취가 심하게 나는 일부 양돈농장 이런 쪽에 악취포집차량을 활용할 계획은 없나요? 저희가 알기로는 민원이 있는 양돈농가에 환경과에서 배치해서 포집한 것으로 알고 있습니다. 민원이 접수되면 배치할 수 있다 이런 얘기죠? 자세한 계획까지는 저희 과에서는 잘 모르겠습니다. 예, 환경과에서……. 어쨌든 양돈농가에서 발생되는 악취로 인한 민원이 끊이지 않는 게 현실인데 지속적으로 냄새가 덜 날 수 있게끔 미생물이나 이런 것을 축산식품과에서도 많이 신경 쓰고 있는 거죠? 예, 그렇습니다. 그런 쪽으로 해서 냄새가 덜 날 수 있게 노력을 해 주시길 바라면서, 우뜸한우사업이라는 걸 알고 계시죠? 참우뜸사업 알고 있습니다. 참우뜸사업. 그 부분에 대해서는 여기 거론이 전혀 안 돼 있는데 예산을 승인해 줬던 것으로 기억하는데 현재 사업이 어떻게 진행되고 있으며 예산부분이나 이런 게 모자라지는 않는지 이런 부분도 한번 설명을 해 주시기 바랍니다. 지금 참우뜸사업은 축협에서 진행을 하고 있습니다. 저희가 축협 이름으로 해서 하고 있는데 사업비는 최상기 팀장님이 자세한 것 좀 설명해 주시기 바랍니다. 축산식품과 농축산식품팀장 최상기입니다. 방금 의원님께서 말씀하신 참우뜸사업은 당초 저희가 군 50%, 축협 50% 해서 총 4,800만원 예산가지고 1년간 관내 소 도축분에 대해서 두당 20만원씩 지

In [48]:
test_dataset[199]["input_ids"][1]

array([14074, 16469, 14581, 13590, 14262, 14426, 14033, 17054, 14543,
       14955, 14549, 14734, 14340, 27269, 14025, 19050, 20706, 15693,
       15118, 11264, 14199, 21993,  9173, 28640,  9866, 22804, 15543,
       15693, 14466, 14032, 15489,  9475, 14605, 17939, 14088, 21888,
        9264, 25490, 14562, 14487, 14285, 17149, 14246, 21763, 18193,
       14303, 14214, 11911, 10184, 13590, 11911, 14025, 12471,  9866,
       14972, 14056, 19867, 14303, 14596, 13676, 25899, 22245, 12007,
       15709, 16467, 14088, 14543, 22456, 21702, 18656, 15176, 14938,
       16334, 14605, 14669, 14528, 15332, 14199, 29072, 17208,  9120,
       14030, 14605, 23928, 29334, 14034, 14704, 14088, 21888,  9264,
       25490, 14562, 14334, 14177, 29883, 14333, 16669, 14153, 25925,
       26638, 14410,  9743, 14158, 11478, 27879, 14092, 13744, 11265,
       14410,  9743, 25925, 13607, 21888,  9264, 25490, 14562, 23433,
        9743, 16345, 23111, 14490, 18940, 15217,  9102, 14032,  9102,
       13586, 11478,